In [124]:
import importlib

from keras.layers import Input, Embedding, GRU, GRUCell, Bidirectional, TimeDistributed, Dense, Lambda
from keras.models import Model, load_model
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
import keras.backend as K
from keras.utils import plot_model

import numpy as np
import random

from qgen.data import training_data, test_data, collapse_documents, expand_answers, _read_data

import qgen.utils as utils
from qgen.embedding import glove

import matplotlib.pyplot as plt

In [125]:
#### PARAMETERS ####

VOCAB_SIZE = glove.shape[0]
EMBEDDING_DIMENS = glove.shape[1]

GRU_UNITS = 100
MAX_DOC_SIZE = None
MAX_ANSWER_SIZE = None
MAX_Q_SIZE = None


In [126]:
#### TRAINING MODEL ####

document_tokens = Input(shape=(MAX_DOC_SIZE,), name="document_tokens")

embedding = Embedding(input_dim = VOCAB_SIZE, output_dim = EMBEDDING_DIMENS, weights=[glove], mask_zero = True, name = 'embedding')
document_emb = embedding(document_tokens)

answer_outputs = Bidirectional(GRU(GRU_UNITS, return_sequences=True), name = 'answer_outputs')(document_emb)
answer_tags = Dense(1, activation = 'sigmoid', name = 'answer_tags')(answer_outputs)

encoder_input_mask = Input(shape=(MAX_ANSWER_SIZE, MAX_DOC_SIZE), name="encoder_input_mask")
encoder_inputs = Lambda(lambda x: K.batch_dot(x[0], x[1]), name="encoder_inputs")([encoder_input_mask, answer_outputs])
encoder_cell = GRU(2 * GRU_UNITS, name = 'encoder_cell')(encoder_inputs)

decoder_inputs = Input(shape=(MAX_Q_SIZE,), name="decoder_inputs")
decoder_emb = embedding(decoder_inputs)
decoder_emb.trainable = False
decoder_cell = GRU(2 * GRU_UNITS, return_sequences = True, name = 'decoder_cell')
decoder_states = decoder_cell(decoder_emb, initial_state = [encoder_cell])

decoder_projection = Dense(VOCAB_SIZE, name = 'decoder_projection', activation = 'softmax')
decoder_outputs = decoder_projection(decoder_states)

total_model = Model([document_tokens, decoder_inputs, encoder_input_mask], [answer_tags, decoder_outputs])
plot_model(total_model, to_file='model.png',show_shapes=True)

In [127]:
#### INFERENCE MODEL ####

decoder_inputs_dynamic = Input(shape=(1,), name="decoder_inputs_dynamic")
decoder_emb_dynamic = embedding(decoder_inputs_dynamic)
decoder_init_state_dynamic = Input(shape=(2 * GRU_UNITS,), name = 'decoder_init_state_dynamic') #the embedding of the previous word
decoder_states_dynamic = decoder_cell(decoder_emb_dynamic, initial_state = [decoder_init_state_dynamic])
decoder_outputs_dynamic = decoder_projection(decoder_states_dynamic)

answer_model = Model(document_tokens, [answer_tags])
decoder_initial_state_model = Model([document_tokens, encoder_input_mask], [encoder_cell])
question_model = Model([decoder_inputs_dynamic, decoder_init_state_dynamic], [decoder_outputs_dynamic, decoder_states_dynamic])



In [128]:
#### COMPILE TRAINING MODEL ####

opti = Adam(lr=0.01)
total_model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy']
                    , optimizer=opti
                    , loss_weights = [0.5, 0.5]) 


In [129]:
#### LOAD DATA ####

training_data_gen = training_data()
# training_data_gen = [next(training_data_gen)]
test_data_gen = test_data()

training_loss_history = []
test_loss_history = []


In [133]:
### LOAD OLD MODELS ####

model_num = 19

total_model.load_weights('./saved_models/keras_{}.h5'.format(model_num), by_name = True)
question_model.load_weights('./saved_models/keras_{}.h5'.format(model_num), by_name = True)
answer_model.load_weights('./saved_models/keras_{}.h5'.format(model_num), by_name = True)
decoder_initial_state_model.load_weights('./saved_models/keras_{}.h5'.format(model_num), by_name = True)

In [134]:
#### GET TEST BATCH

batch = next(test_data_gen)
batch = collapse_documents(batch)

In [135]:
#### TESTING ####

importlib.reload(utils)

idx = 0
idx = random.randint(0,10)
print(idx)

answer_preds = answer_model.predict(batch["document_tokens"])

answers = np.squeeze(np.round(answer_preds))
answer_batch = expand_answers(batch, answers)

utils.show_test(batch, answer_batch, idx, answer_model, decoder_initial_state_model, question_model)

#### PLOT ANSWER LIKELIHOOD ####
plt.plot(batch['answer_labels'][idx])
plt.plot(answer_preds[idx])
plt.show()

8

 TEXT
-LRB- CNN -RRB- -- An Australian pilot landed a company plane in the ocean off Darwin on Friday , in a water landing that prompted comparisons to last month 's landing of a US Airways jet in New York 's Hudson River . The Australian pilot has been compared to U.S. hero Chesley B. `` Sully '' Sullenberger , pictured above . The Australian pilot also encountered problems shortly after takeoff . The twin-propeller plane was carrying the pilot and five passengers when it went down shortly after leaving Darwin International Airport , said Neville Blyth , a senior transport safety investigator with the Australian Transport Safety Bureau . All those aboard were unhurt and managed to walk to shore , he said . Pictures showed the men from the plane soaked and carrying bags and equipment as they walked through the water . `` It 's essentially a good story , '' Blyth said . The ATSB is deciding whether to investigate , he said , and the cause of the incident is still unknown . The Piper 

InvalidArgumentError: Tried to read from index -1 but array size is: 0
	 [[{{node encoder_cell_1/TensorArrayReadV3}} = TensorArrayReadV3[dtype=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](encoder_cell_1/TensorArray, encoder_cell_1/sub_1, encoder_cell_1/while/Exit_1)]]

In [136]:
answer_batch['document_tokens'][[idx]]

array([[   1, 4633,    1,   66,   30,  800, 2465, 3723,   11,  128, 1304,
          10,    4, 2527,  138,    1,   16,  185,    5,   10,   11,  429,
        3382,   15, 3448,    1,    8,   77,  228,   12, 3382,    7,   11,
          96, 4982, 3363,   10,   51,  196,   12, 5567,  619,    6,    4,
         800, 2465,   32,   52, 1120,    8,   99, 3573,    1, 3212,   29,
           1,   28, 9879,    5, 9601, 1063,    6,    4,  800, 2465,   53,
        6984,  670, 1837,   50, 9123,    6,    4,    1, 1304,   18, 1760,
           4, 2465,    9,  174, 1869,   62,   23,  387,  135, 1837,   50,
        1213,    1,  146,  917,    5,   19,    1,    1,    5,   11,  597,
        1855, 1082, 7256,   20,    4,  800, 1855, 1082, 1781,    6,   65,
         155, 3393,   36,    1,    9, 1745,    8, 2354,    8, 4024,    5,
          21,   19,    6, 2457,  891,    4,  300,   26,    4, 1304,    1,
           9, 1760, 4953,    9, 1392,   22,   40, 2983,  131,    4,  429,
           6,   29,   23,   12, 4506, 

In [138]:
answer_batch['answer_masks'][[idx]]

array([], shape=(1, 0, 200), dtype=int32)

In [137]:
next_decoder_init_state = decoder_initial_state_model.predict([answer_batch['document_tokens'][[idx]], answer_batch['answer_masks'][[idx]]])

InvalidArgumentError: Tried to read from index -1 but array size is: 0
	 [[{{node encoder_cell_1/TensorArrayReadV3}} = TensorArrayReadV3[dtype=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](encoder_cell_1/TensorArray, encoder_cell_1/sub_1, encoder_cell_1/while/Exit_1)]]